# ETL Process for PC PARTS Dataset

This notebook performs a *semi automated ETL* process for sales, cost, and inventory data stored in the `PCPARTS_DB` database, with human validation. The database contains a mix of real and synthetic data. Once processed, the data is loaded into SQL Server and ready for use.

## Performed Steps

1. **Reading datasets** from local sources or previous transformations.
2. **Connecting to SQL Server** using SQLAlchemy and ODBC.
3. **Uploading dataframes** to the database:
   - `df_ventas`
   - `df_costos`
   - `df_inventario`
   - `inventario_agregado`
   - `ventas_agregado`

> *Connection credentials have been censored for security purposes.*

> *This an ongoing project building and further refinement/improvement is pending to be implemented*


In [4]:
# Ruta de los archivos CSV
ruta = r"C://Users//PC USER//PC PART DB//Current month"

In [6]:
import os
import pandas as pd
import unicodedata


# Dictionary
dataframes = {}

# Function to normalize text
def normalizar(texto):
    return unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('utf-8').lower().strip()

# Loading and cleaning files name 
for archivo in os.listdir(ruta):
    if archivo.endswith('.csv'):
        nombre_df = archivo.replace('.csv', '').replace('-', '').replace(' ', '').replace('_', '')
        nombre_df = normalizar(nombre_df).capitalize()

        df = pd.read_csv(os.path.join(ruta, archivo))

# Columns name cleaning
        nuevas_columnas = []
        for col in df.columns:
            col = normalizar(col)
            col = col.capitalize()
            col = col.replace(' ', '_').replace('-', '_')

            if col == "Fecha":
                if "inventario" in nombre_df:
                    col = f"{col}_inventario"
                elif "venta" in nombre_df:
                    col = f"{col}_venta"
                elif "costo" in nombre_df:
                    col = f"{col}_costo"
                
            nuevas_columnas.append(col)
        df.columns = nuevas_columnas

# Columns name cleaning

        for col in df.select_dtypes(include='object').columns:
            df[col] = df[col].apply(normalizar).str.capitalize()
            if col.startswith("Fecha_"):
                df[col] = pd.to_datetime(df[col], errors='coerce')  

        dataframes[nombre_df] = df  
        
# Groups by table type
ventas = []
inventario = []
costos = []

for nombre, df in dataframes.items():
    if "inventario" in nombre:
        inventario.append(df)
    elif "venta" in nombre:
        ventas.append(df)
    elif "costo" in nombre:
        costos.append(df)

# Append tables
df_ventas = pd.concat(ventas, ignore_index=True)
df_inventario = pd.concat(inventario, ignore_index=True)
df_costos = pd.concat(costos, ignore_index=True)

TablasT = [ventas, inventario, costos]

for z in TablasT:
    for w in z:
        if w.isnull().sum().sum()>0:
            print(f"NULLS IN {w.isnull().sum().sum()}")


# Aggregated Inventory

inventario_agregado = df_inventario.groupby([
    pd.Grouper(key='Fecha_inventario', freq='ME'),
    'Tienda_id', 'Sku'
]).agg({
    'Stock_inicial': 'sum',    
    'Stock_final': 'sum'       
}).reset_index()

# Aggregated Sales

ventas_agregado = df_ventas.groupby([
    pd.Grouper(key='Fecha_venta', freq='ME'), 
    'Tienda_id', 'Sku'
]).agg({
    'Unidades_vendidas': 'sum',
    'Ingresos': 'sum'
}).reset_index()

# For human validation before upload

print("Inventory\n")
print(df_inventario.head(), "\n")

print("Sales\n")
print(df_ventas.head(), "\n")

print("Costs\n")
print(df_costos.head(), "\n")

print("Aggregated Inventory\n")
print(inventario_agregado.head(), "\n")

print("Aggregated Sales\n")
print(ventas_agregado.head(), "\n")


Inventory

  Fecha_inventario       Sku  Stock_inicial  Stock_recibido  Stock_vendido  \
0       2023-04-01  Sku_0057             55              32             39   
1       2023-04-08  Sku_0057             93              45            102   
2       2023-04-15  Sku_0057             40              50             39   
3       2023-04-22  Sku_0057             36              15             29   
4       2023-04-01  Sku_0090             29               2              5   

   Stock_final  Descontinuado Tienda_id  
0           48          False  Tienda_1  
1           36          False  Tienda_1  
2           51          False  Tienda_1  
3           22          False  Tienda_1  
4           26          False  Tienda_1   

Sales

  Fecha_venta       Sku  Precio_unitario  Unidades_vendidas   Ingresos  \
0  2023-04-03  Sku_0053          9875.22                 26  256755.72   
1  2023-04-03  Sku_0085          9204.58                  2   18409.16   
2  2023-04-03  Sku_0048          4178

In [34]:
# Upload to SQL

from sqlalchemy import create_engine
from urllib.parse import quote_plus

cadena_odbc = (
    "*********"
)

cadena_conexion = "*********" + quote_plus(cadena_odbc)
engine = create_engine(cadena_conexion)

df_ventas.to_sql('df_ventas', engine, if_exists='append', index=False)
df_costos.to_sql('df_costos', engine, if_exists='append', index=False)
df_inventario.to_sql('df_inventario', engine, if_exists='append', index=False)
inventario_agregado.to_sql('inventario_agregado', engine, if_exists='append', index=False)
ventas_agregado.to_sql('ventas_agregado', engine, if_exists='append', index=False)

